In [1]:
import os
import json
import numpy as np
import cv2
import pandas as pd
import time

In [2]:
def pad(img):
    max_size = max(img.shape)
    top = int((max_size-img.shape[0])/2)
    bot = int((max_size-img.shape[0])/2)
    left = int((max_size-img.shape[1])/2)
    right = int((max_size-img.shape[1])/2)
    return cv2.copyMakeBorder(img, top, bot, left, right, cv2.BORDER_CONSTANT, None, value = 0)

def center_crop(img, w, h):
    center = img.shape
    x = center[1]/2 - w/2
    y = center[0]/2 - h/2

    return img[int(y):int(y+h), int(x):int(x+w)]

def update_json(word, video_id, frame_count, json_file, class_list, subset='train'):
    label = class_list.index[class_list[1] == word].tolist()[0]
    vid_info = {'subset': subset, 'action': [label, 1, frame_count]}
    video_id = 'additional_data'+ "\ ".strip() + video_id
    json_file.update({video_id:vid_info})

<h1>Instructions</h1>

- Delete "DELETE_THIS" in the "additional_data" folder
- Make sure to enter the word you want to record into the "word" variable before you record
- After finishing recording, zip up the "additional_data" folder and "nslt_add_data.json" file and upload to the "additional_data" folder on Google drive
- Key binding: 'o' for start recording, 'p' for stop recording; during replay: 'd' for discard recording, 's' for save recording, 'q' for quit

In [3]:
dir = os.listdir("additional_data")
if len(dir) == 0:
    new_filename = 0
else:
    filenames = []
    for file in os.listdir("additional_data"):
        filenames += [int(file[2:-4])]
    new_filename = max(filenames)+1

In [4]:
word = "ENTER WORD YOU WANT TO RECORD"


#Initailize capture param
cap = cv2.VideoCapture(0)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
short_side = int(min(width, height))
long_side = int((short_side*4)/3)

#Initialize video writer param
resolution = tuple((256,256))
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
frame_count = 0
replay_flag = 0
loop_flag = 0

#Initialize timer param
tick = time.time()
output_string = "Press \"o\" to record"
countdown_flag = 0
record_flag = 0
sec = ""

#Initialize json param
f = open('nslt_add_data.json')
json_file = json.load(f)
class_list = pd.read_csv("wlasl_class_list.txt", sep='\t', header=None)
root = "additional_data/"


while cap.isOpened():
    
    #Wait for key press
    k = cv2.waitKey(10) & 0xFF
    
    # Read feed
    ret, frame = cap.read()

    if ret == 0:
        break
        
    if loop_flag==1:
        ret, frame = cap2.read()
        if not ret:
            cap2.set(cv2.CAP_PROP_POS_FRAMES, 0)
            continue
        frame = cv2.resize(frame, dsize=(512,512))
    
    #Frame processing
    output_frame = frame.copy()
    frame = center_crop(frame, long_side, short_side)
    frame = pad(frame)
    frame = cv2.resize(frame, dsize=(256, 256))
    
    #Press o to start countdown and recording
    if (countdown_flag+record_flag == 0) and k == ord('o'):
        tick = time.time()
        sec = 3
        output_string = ""
        countdown_flag = 1
        
    #Press p to stop recording
    if (record_flag == 1) and (k == ord('p')):
        out.release()
        sec = ""
        replay_flag = 1
        record_flag = 0
        
    #Replay
    if (replay_flag == 1):
        cap2 = cv2.VideoCapture(root + video_id + '.mp4')
        output_string = "Replay"
        replay_flag = 0
        loop_flag = 1
        
    #Press s to save
    if (loop_flag == 1) and (k == ord('s')):
        cap2.release()
        update_json(word, video_id, frame_count, json_file, class_list)
        new_filename += 1
        frame_count = 0
        output_string = "Press \"o\" to record"
        loop_flag = 0
        
    #Press d to discard
    if (loop_flag == 1) and (k == ord('d')):
        cap2.release()
        os.remove(root + video_id + '.mp4')
        frame_count = 0
        output_string = "Press \"o\" to record"
        loop_flag = 0
        
    tock = time.time()*countdown_flag
    
    #Countdown timer
    if tock-tick>=1:
        tick = time.time()
        sec -= 1
        #When countdown ends start recording
        if sec<1:
            video_id = 't_'+str(new_filename)
            out = cv2.VideoWriter(root + video_id + '.mp4', fourcc, 25.0, resolution)
            sec = ""
            output_string = 'Recording... press \"p\" to stop'
            frame_count = 0
            countdown_flag = 0
            record_flag = 1
    
    #Render text to feed
    cv2.rectangle(output_frame, (0,0), (640, 40), (245, 117, 16), -1)
    cv2.putText(output_frame, output_string + str(sec), (3,30), 
    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
    #Write framems to file when recording activated
    if record_flag==1:
        out.write(frame)
        frame_count += 1
        
    #Show camera feed
    cv2.imshow('OpenCV Feed', output_frame)

    # Break
    if k == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()
with open("nslt_add_data.json", "w") as outfile:
        json.dump(json_file, outfile)